##CUNY - Data 612

#Final Project
##Violeta Stoyanova
##Peter Kowalchuk

#Introduction

“For those of us fortunate enough to be healthy and working from home during the outbreak of COVID-19, the possibilities for entertainment can feel both extremely confined (specifically, to our homes) and overwhelmingly endless (where in our Netflix queues to begin?) (Cohen, 2020).” Now more than ever in our socially-distanced life we depend on recommendations for movies to fill our quarantine days with entertainment. Some rely on movie experts; others look at ratings to pick flicks and cure some of the overwhelming boredom. In this regard, for our final project we will attempt to build a recommender system with the use of ‘implicit’ movie data for the model to recommend new movies to users. 

#Objective

The goal of this project is to build various models in the attempt to recommend new movies to users based on ‘implicit’ data. We will be using a dataset from kaggle.com that contains descriptions of 34,886 movies from around the world. This dataset contains a summary of the plot of each movie. The goal is to use this plot to generate recommendations based on the works present in the plot. A first implementation of the recommender will provide a recommendation based on a word or list of words provided. One can imagine how a more complex implementation of this approach could allow the user write a plot of a movie she/he would like to watch. The system then would provide recommendations based on this plot by using the list of words contained in it as input to the recommender system presented in this project.

#Methodology

For the purposes of this project we will utilize the speed and scalability of Apache Spark via Databricks. The first step will be to tokenize the data, so we are able extract some weight with the use of the ‘genre’ variable. Via TF-IDF we will first eliminate any stop words and count word frequency and the build a utility matrix. With this approach words will be equivalent to users in a conventional recommender, with the word weight provided by TF-IDF representing the ratings for each movie. Afterwards, using the techniques we have learned and applied in class, such as Item-based collaborative filtering and User-based collaborative filtering (IBCF and UBCF, respectively) and ALS (Alternating Least Squares) we will build recommender systems in order for movies to be recommended to users. 
We believe that proper evaluation is important to detect how accurate our algorithms are at recommending movies thus we will root mean square error metric (RMSE).

#Data

The dataset contains descriptions of 34,886 movies from around the world. Column descriptions are listed below:
  - Release Year - Year in which the movie was released
  - Title - Movie title
  - Origin/Ethnicity - Origin of movie (i.e. American, Bollywood, Tamil, etc.)
  - Director - Director(s)
  - Plot - Main actor and actresses
  - Genre - Movie Genre(s)
  - Wiki Page - URL of the Wikipedia page from which the plot description was scraped
  - Plot - Long form description of movie plot (WARNING: May contain spoilers!!!)

In [3]:
# File location and type
file_location = "/FileStore/tables/wiki_movie_plots_deduped.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

We load the data and observe how many movie plot we have

In [5]:
df.count()

Out[2]: 134113

We clean the dataframe to only include the columns we need and delete all row without movie plots

In [7]:
df=df.drop("_c0","_c2","_c3","_c4","_c6")

df=df.selectExpr("_c1 as Title","_c5 as Genre","_c7 as Plot")
df = df.filter(df.Title != "Title")
df = df.dropna()
display(df)

Title,Genre,Plot
Kansas Saloon Smashers,unknown,"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"
Love by the Light of the Moon,unknown,"The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better."
The Martyred Presidents,unknown,"The film, just over a minute long, is composed of two shots. In the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. At the center of the altar, a viewing portal displays the portraits of three U.S. Presidents—Abraham Lincoln, James A. Garfield, and William McKinley—each victims of assassination."
"Terrible Teddy, the Grizzly King",unknown,"""Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading """"His Photographer"""" and """"His Press Agent"""" respectively"
Jack and the Beanstalk,unknown,"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. The giant chases Jack down the bean stalk, but Jack is able to cut it down before the giant can get to safety. He falls and is killed as Jack celebrates. The fairy then reveals that Jack may return home as a prince."
Alice in Wonderland,unknown,"""Alice follows a large white rabbit down a """"Rabbit-hole"""". She finds a tiny door. When she finds a bottle labeled """"Drink me"""""
The Great Train Robbery,western,"The film opens with two bandits breaking into a railroad telegraph office, where they force the operator at gunpoint to have a train stopped and to transmit orders for the engineer to fill the locomotive's tender at the station's water tank. They then knock the operator out and tie him up. As the train stops it is boarded by the bandits‍—‌now four. Two bandits enter an express car, kill a messenger and open a box of valuables with dynamite; the others kill the fireman and force the engineer to halt the train and disconnect the locomotive. The bandits then force the passengers off the train and rifle them for their belongings. One passenger tries to escape but is instantly shot down. Carrying their loot, the bandits escape in the locomotive, later stopping in a valley where their horses had been left."
The Suburbanite,comedy,"The film is about a family who move to the suburbs, hoping for a quiet life. Things start to go wrong, and the wife gets violent and starts throwing crockery, leading to her arrest."
The Little Train Robbery,unknown,"""The opening scene shows the interior of the robbers' den. The walls are decorated with the portraits of notorious criminals and pictures il

Now we have a reduced dataframe with only movies and plot

In [9]:
df.count()

Out[4]: 69981

The next step is to get rid of punctuation and capitalization

In [11]:
from pyspark.sql.functions import regexp_replace, trim, col, lower

In [12]:
def removePunctuation(column):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        column (Column): A Column containing a sentence.

    Returns:
        Column: A Column named 'sentence' with clean-up operations applied.
    """
 #df = sqlContext.sql(SELECT * FROM table)
    return lower(trim(regexp_replace(column,'[^A-Za-z0-9 ]+', ''))).alias('PlotNew')

In [13]:
plotDf= df.select(removePunctuation(col('Plot')))

In [14]:
display(plotDf)

PlotNew
a bartender is working at a saloon serving drinks to customers after he fills a stereotypically irish mans bucket with beer carrie nation and her followers burst inside they assault the irish man pulling his hat over his eyes and then dumping the beer over his head the group then begin wrecking the bar smashing the fixtures mirrors and breaking the cash register the bartender then sprays seltzer water in nations face before a group of policemen appear and order everybody to leave1
the moon painted with a smiling face hangs over a park at night a young couple walking past a fence learn on a railing and look up the moon smiles they embrace and the moons smile gets bigger they then sit down on a bench by a tree the moons view is blocked causing him to frown in the last scene the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better
the film just over a minute long is composed of two shots in the first a girl sits at the base of an altar or tomb her face hidden from the camera at the center of the altar a viewing portal displays the portraits of three us presidentsabraham lincoln james a garfield and william mckinleyeach victims of assassination
lasting just 61 seconds and consisting of two shots the first shot is set in a wood during winter the actor representing then vicepresident theodore roosevelt enthusiastically hurries down a hillside towards a tree in the foreground he falls once but rights himself and cocks his rifle two other men bearing signs reading his photographer and his press agent respectively
the earliest known adaptation of the classic fairytale this films shows jack trading his cow for the beans his mother forcing him to drop them in the front yard and beig forced upstairs as he sleeps jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk in this version jack is the son of a deposed king when jack wakes up he finds the beanstalk has grown and he climbs to the top where he enters the giants home the giant finds jack who narrowly escapes the giant chases jack down the bean stalk but jack is able to cut it down before the giant can get to safety he falls and is killed as jack celebrates the fairy then reveals that jack may return home as a prince
alice follows a large white rabbit down a rabbithole she finds a tiny door when she finds a bottle labeled drink me
the film opens with two bandits breaking into a railroad telegraph office where they force the operator at gunpoint to have a train stopped and to transmit orders for the engineer to fill the locomotives tender at the stations water tank they then knock the operator out and tie him up as the train stops it is boarded by the banditsnow four two bandits enter an express car kill a messenger and open a box of valuables with dynamite the others kill the fireman and force the engineer to halt the train and disconnect the locomotive the bandits then force the passengers off the train and rifle them for their belongings one passenger tries to escape but is instantly shot down carrying their loot the bandits escape in the locomotive later stopping in a valley where their horses had been left
the film is about a family who move to the suburbs hoping for a quiet life things start to go wrong and the wife gets violent and starts throwing crockery leading to her arrest
the opening scene shows the interior of the robbers den the walls are decorated with the portraits of notorious criminals and pictures illustrating the exploits of famous bandits some of the gang are lounging about while others are reading novels and illustrated papers although of youthful appearance each is dressed like a typical western desperado the bandit queen
and are obliged to take chances on foot the police now get in sight of the fleeing robbers and a lively chase follows through tall weeds


In [15]:
from pyspark.sql.types import StructType

In [16]:
#merging the two dataframes

df3_schema = StructType(df.schema.fields + plotDf.schema.fields)   

def mergeDataFrames(x):
  dt1 = x[0]
  dt2 = x[1]
  Title = dt1[0]
  Genre = dt1[1]
  Plot = dt1[2]
  PlotNew = dt2[0]
  return [Title,Genre,Plot,PlotNew]

rdd_merged = df.rdd.zip(plotDf.rdd).map(lambda x: mergeDataFrames(x))
df3 = spark.createDataFrame(rdd_merged, schema=df3_schema)

In [17]:
display(df3)

Title,Genre,Plot,PlotNew
Kansas Saloon Smashers,unknown,"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]",a bartender is working at a saloon serving drinks to customers after he fills a stereotypically irish mans bucket with beer carrie nation and her followers burst inside they assault the irish man pulling his hat over his eyes and then dumping the beer over his head the group then begin wrecking the bar smashing the fixtures mirrors and breaking the cash register the bartender then sprays seltzer water in nations face before a group of policemen appear and order everybody to leave1
Love by the Light of the Moon,unknown,"The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better.",the moon painted with a smiling face hangs over a park at night a young couple walking past a fence learn on a railing and look up the moon smiles they embrace and the moons smile gets bigger they then sit down on a bench by a tree the moons view is blocked causing him to frown in the last scene the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better
The Martyred Presidents,unknown,"The film, just over a minute long, is composed of two shots. In the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. At the center of the altar, a viewing portal displays the portraits of three U.S. Presidents—Abraham Lincoln, James A. Garfield, and William McKinley—each victims of assassination.",the film just over a minute long is composed of two shots in the first a girl sits at the base of an altar or tomb her face hidden from the camera at the center of the altar a viewing portal displays the portraits of three us presidentsabraham lincoln james a garfield and william mckinleyeach victims of assassination
"Terrible Teddy, the Grizzly King",unknown,"""Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during winter. The actor representing then vice-president Theodore Roosevelt enthusiastically hurries down a hillside towards a tree in the foreground. He falls once, but rights himself and cocks his rifle. Two other men, bearing signs reading """"His Photographer"""" and """"His Press Agent"""" respectively",lasting just 61 seconds and consisting of two shots the first shot is set in a wood during winter the actor representing then vicepresident theodore roosevelt enthusiastically hurries down a hillside towards a tree in the foreground he falls once but rights himself and cocks his rifle two other men bearing signs reading his photographer and his press agent respectively
Jack and the Beanstalk,unknown,"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow for the beans, his mother forcing him to drop them in the front yard, and beig forced upstairs. As he sleeps, Jack is visited by a fairy who shows him glimpses of what will await him when he ascends the bean stalk. In this version, Jack is the son of a deposed king. When Jack wakes up, he finds the beanstalk has grown and he climbs to the top where he enters the giant's home. The giant finds Jack, who narrowly escapes. Th

#TF-IDF

The next step is to remove stop words and tokenize the data in order to create 'weights' from the words in the Plot column

In [20]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer

In [21]:
#tokenize
tokenizer = Tokenizer(inputCol="PlotNew", outputCol="words")
wordsData = tokenizer.transform(df3)

#remove stop words
remover = StopWordsRemover(inputCol="words",outputCol="noStopWords")
wordsData = remover.transform(wordsData)
wordsData = wordsData.drop("words")
wordsData=wordsData.selectExpr("Title as Title","Genre as Genre","noStopWords as words")
display(wordsData)

Title,Genre,words
Kansas Saloon Smashers,unknown,"List(bartender, working, saloon, serving, drinks, customers, fills, stereotypically, irish, mans, bucket, beer, carrie, nation, followers, burst, inside, assault, irish, man, pulling, hat, eyes, dumping, beer, head, group, begin, wrecking, bar, smashing, fixtures, mirrors, breaking, cash, register, bartender, sprays, seltzer, water, nations, face, group, policemen, appear, order, everybody, leave1)"
Love by the Light of the Moon,unknown,"List(moon, painted, smiling, face, hangs, park, night, young, couple, walking, past, fence, learn, railing, look, moon, smiles, embrace, moons, smile, gets, bigger, sit, bench, tree, moons, view, blocked, causing, frown, last, scene, man, fans, woman, hat, moon, left, sky, perched, shoulder, see, everything, better)"
The Martyred Presidents,unknown,"List(film, minute, long, composed, two, shots, first, girl, sits, base, altar, tomb, face, hidden, camera, center, altar, viewing, portal, displays, portraits, three, us, presidentsabraham, lincoln, james, garfield, william, mckinleyeach, victims, assassination)"
"Terrible Teddy, the Grizzly King",unknown,"List(lasting, 61, seconds, consisting, two, shots, first, shot, set, wood, winter, actor, representing, vicepresident, theodore, roosevelt, enthusiastically, hurries, hillside, towards, tree, foreground, falls, rights, cocks, rifle, two, men, bearing, signs, reading, photographer, press, agent, respectively)"
Jack and the Beanstalk,unknown,"List(earliest, known, adaptation, classic, fairytale, films, shows, jack, trading, cow, beans, mother, forcing, drop, front, yard, beig, forced, upstairs, sleeps, jack, visited, fairy, shows, glimpses, await, ascends, bean, stalk, version, jack, son, deposed, king, jack, wakes, finds, beanstalk, grown, climbs, top, enters, giants, home, giant, finds, jack, narrowly, escapes, giant, chases, jack, bean, stalk, jack, able, cut, giant, get, safety, falls, killed, jack, celebrates, fairy, reveals, jack, may, return, home, prince)"
Alice in Wonderland,unknown,"List(alice, follows, large, white, rabbit, rabbithole, finds, tiny, door, finds, bottle, labeled, drink)"
The Great Train Robbery,western,"List(film, opens, two, bandits, breaking, railroad, telegraph, office, force, operator, gunpoint, train, stopped, transmit, orders, engineer, fill, locomotives, tender, stations, water, tank, knock, operator, tie, train, stops, boarded, banditsnow, four, two, bandits, enter, express, car, kill, messenger, open, box, valuables, dynamite, others, kill, fireman, force, engineer, halt, train, disconnect, locomotive, bandits, force, passengers, train, rifle, belongings, one, passenger, tries, escape, instantly, shot, carrying, loot, bandits, escape, locomotive, later, stopping, valley, horses, left)"
The Suburbanite,comedy,"List(film, family, move, suburbs, hoping, quiet, life, things, start, go, wrong, wife, gets, violent, starts, throwing, crockery, leading, arrest)"
The Little Train Robbery,unknown,"List(opening, scene, shows, interior, robbers, den, walls, decorated, portraits, notorious, criminals, pictures, illustrating, exploits, famous, bandits, gang, lounging, others, reading, novels, illustrated, papers, although, youthful, appearance, dressed, like, typical, western, desperado, bandit, queen)"
where the remainder of the gang have been waiting for them. Believing they have successfully eluded their pursuers,abandoning everything,"List(obliged, take, chances, foot, police, get, sight, fleeing, robbers, lively, chase, follows, tall, weeds)"


In [22]:
#TF
#hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
#featurizedData = hashingTF.transform(wordsData)
cv = CountVectorizer(inputCol="words", outputCol="rawFeatures", minDF=10.0)
model = cv.fit(wordsData)
vocabulary=model.vocabulary #we will use this to map back our words when using the recommender
featurizedData = model.transform(wordsData)

#IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

#TF-IDF
rescaledData = idfModel.transform(featurizedData)

display(rescaledData.select("Title", "words","features"))

Title,words,features
Kansas Saloon Smashers,"List(bartender, working, saloon, serving, drinks, customers, fills, stereotypically, irish, mans, bucket, beer, carrie, nation, followers, burst, inside, assault, irish, man, pulling, hat, eyes, dumping, beer, head, group, begin, wrecking, bar, smashing, fixtures, mirrors, breaking, cash, register, bartender, sprays, seltzer, water, nations, face, group, policemen, appear, order, everybody, leave1)","List(0, 17164, List(6, 87, 135, 140, 142, 307, 362, 420, 564, 620, 890, 1192, 1348, 1429, 1435, 1596, 2288, 2565, 2919, 3071, 3113, 3114, 3200, 3736, 3786, 4074, 4654, 5271, 5290, 5922, 6167, 7174, 8580, 9697, 9999, 10923, 11924, 12501, 13051), List(2.6888310872967875, 7.529156220864204, 3.934157519819011, 3.9608060249287513, 3.9920466607649128, 4.585110382767875, 4.628035427484909, 4.700794781767338, 5.048970457365206, 5.142278189064658, 5.359935594342088, 5.60691726021224, 5.675354421765468, 5.713575634585666, 5.7623657987550985, 5.818455265406142, 12.501437133338062, 6.280796021906308, 6.4738621179832405, 6.56087349497287, 6.483164510645554, 6.571025866436888, 6.623393851954204, 6.786545492640439, 6.749274097843207, 6.907498103058101, 6.9364856399313535, 14.528346093993667, 7.418323726824092, 7.327351948618365, 14.65470389723673, 7.492431698977813, 7.7547959634453045, 7.860156479103131, 7.977939514759514, 8.02049912917831, 8.16026107155347, 8.21155436594102, 8.322780001051244))"
Love by the Light of the Moon,"List(moon, painted, smiling, face, hangs, park, night, young, couple, walking, past, fence, learn, railing, look, moon, smiles, embrace, moons, smile, gets, bigger, sit, bench, tree, moons, view, blocked, causing, frown, last, scene, man, fans, woman, hat, moon, left, sky, perched, shoulder, see, everything, better)","List(0, 17164, List(6, 9, 24, 37, 59, 126, 137, 146, 177, 231, 307, 309, 465, 483, 522, 555, 665, 695, 1186, 1341, 1641, 2602, 2907, 3807, 4074, 4315, 4579, 4985, 5171, 5194, 5727, 6861, 7193, 8311, 8486, 8487, 8891, 11686, 13296), List(2.6888310872967875, 2.7335504902370333, 3.1429812347385444, 3.230474365320534, 3.469831041614554, 3.934157519819011, 3.9422250369888183, 4.0810301471414165, 4.126905780957798, 4.332707222751773, 4.585110382767875, 4.529275595858436, 4.796419476435083, 4.855207550444216, 4.879349855765816, 4.913770079652295, 5.215822092387029, 5.10155399883809, 5.748821573647342, 17.806912560087405, 5.832983365969052, 6.311806258648868, 6.5020329949499365, 6.713342088617144, 6.907498103058101, 6.879327226091404, 6.907498103058101, 7.028858960062369, 7.078455901201741, 7.078455901201741, 7.327351948618365, 7.4424212784031525, 7.4424212784031525, 7.6902574423077334, 7.7547959634453045, 7.65948578364098, 7.7547959634453045, 16.64556000210249, 8.322780001051244))"
The Martyred Presidents,"List(film, minute, long, composed, two, shots, first, girl, sits, base, altar, tomb, face, hidden, camera, center, altar, viewing, portal, displays, portraits, three, us, presidentsabraham, lincoln, james, garfield, william, mckinleyeach, victims, assassination)","List(0, 17164, List(3, 4, 27, 45, 60, 211, 234, 262, 307, 454, 919, 1018, 1334, 1546, 1789, 2249, 3356, 3473, 3593, 4941, 5741, 5795, 5874, 6194, 7128, 7551, 8029, 12525), List(2.594400566394538, 2.6442147863927215, 3.2097297015269186, 3.259812736491969, 3.500129327491403, 4.3749357191712805, 4.32296161232126, 4.452805231866598, 4.585110382767875, 4.858884025173524, 5.384552221977444, 5.5762635191212375, 5.7222713415532205, 5.857675978559423, 6.008498868294007, 6.214350922498156, 6.581282366604078, 6.749274097843207, 6.66735697537532, 7.2641730469968335, 7.167009298543186, 7.224167712383134, 7.185701431555338, 7.394793229413898, 7.57247440665135, 15.201290567236093, 7.6006452836180465, 8.21155436594102))"
"Terrible Teddy, the Grizzly King","List(lasting, 61, seconds, consisting, two, shots, first, shot, set, wood, winter, actor, representing, vicepresident, theodore, roosevelt, enthusiastically, hurries, hills

#pySpark dataframe

Now that we have weights for all the words in all the movies, we need to prepare a dataframe for the pySpark ALS recommender. The dataframe for ALS has three columns: Users, Item and Rating. In our case Words represent Users, Movies are the Items and the TF_IDF Weight are the Ratings. So we use the output from TF-IDF to build the dataframe with the three required columns.

To do this we use Koalas, which was introduced into Databricks in 2018. Koalas is a spark implementation of Pandas. We use this becouse of the ease of use of Pandas.

In [24]:
import numpy as np
import pandas as pd
import databricks.koalas as ks

We start by converting the output from TF-IDF into a Koalas dataframe

In [26]:
KrescaledData=ks.DataFrame(rescaledData)
display(KrescaledData.head(10))

,Title,Genre,words,rawFeatures,features
0,Kansas Saloon Smashers,unknown,"[bartender, working, saloon, serving, drinks, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.6888310872967..."
1,Love by the Light of the Moon,unknown,"[moon, painted, smiling, face, hangs, park, ni...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.6888310872967..."
2,The Martyred Presidents,unknown,"[film, minute, long, composed, two, shots, fir...","(0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 2.594400566394538, 2.644214786..."
3,"Terrible Teddy, the Grizzly King",unknown,"[lasting, 61, seconds, consisting, two, shots,...","(0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 5.188801132789076, 0.0, 0.0, 0..."
4,Jack and the Beanstalk,unknown,"[earliest, known, adaptation, classic, fairyta...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,Alice in Wonderland,unknown,"[alice, follows, large, white, rabbit, rabbith...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,The Great Train Robbery,western,"[film, opens, two, bandits, breaking, railroad...","(1.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(2.3479231903429225, 0.0, 0.0, 5.1888011327890..."
7,The Suburbanite,comedy,"[film, family, move, suburbs, hoping, quiet, l...","(0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 2.6442147863927215, 2.789..."
8,The Little Train Robbery,unknown,"[opening, scene, shows, interior, robbers, den...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,where the remainder of the gang have been wai...,abandoning everything,"[obliged, take, chances, foot, police, get, si...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Same as we will use vocabulary to map back words, we create a list to map back movies

In [28]:
movieList = KrescaledData['Title'].tolist()

/databricks/spark/python/pyspark/sql/types.py:1636: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 return pa.schema(fields)

We add the first movie to get the pySPark dataframe we need for ALS started.

In [30]:
thisMovie=KrescaledData['features'][0].toArray()

#get words for this movie
words=np.nonzero(thisMovie)

#get weights for the words in this movie
weights=thisMovie[thisMovie!=0]

#create an initial Pandas dataframe with the words and their weights for the first movie
matrixData = pd.DataFrame()
matrixData['word']= words[0]
matrixData['rating']=weights
matrixData['movie']=0

#convert Pandas dataframe to Koalas to make use of spark going forward
matrix=ks.DataFrame(matrixData)
#matrix

Now that we have a dataframe with one movie, we can add all the other movies in our dataset by following the same steps.

In [32]:
import pandas as pd

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

In [33]:
from multiprocessing.pool import ThreadPool

In [34]:
# allow up to 5 concurrent threads
pool = ThreadPool(5)

# hyperparameters to test out (n_trees)
parameters = [1,7]

# define a function to train a RF model and return metrics 
def mllib_random_forest(cluster,movies):
  thisMovie=KrescaledData['features'][cluster].toArray()
  #get words for this movie
  words=np.nonzero(thisMovie)
  #get weights for the words in this movie
  weights=thisMovie[thisMovie!=0]
  #create an initial Pandas dataframe with the words and their weights for the first movie
  matrixData = pd.DataFrame()
  matrixData['word']= words[0]
  matrixData['rating']=weights
  matrixData['movie']=0
  #convert Pandas dataframe to Koalas to make use of spark going forward
  matrix=ks.DataFrame(matrixData)  
  for i in range(cluster,cluster+5):
    thisMovie=movies[i].toArray()
    #get words for this movie
    words=np.nonzero(thisMovie)
    #get weights for the words in this movie
    weights=thisMovie[thisMovie!=0]
    matrixData = pd.DataFrame()
    matrixData['word']= words[0]
    matrixData['rating']=weights
    matrixData['movie']=i
    matrix=matrix.append(ks.DataFrame(matrixData))
  return [matrix]
  
# run the tasks 
pool.map(lambda cluster: mllib_random_forest(cluster,KrescaledData['features']), parameters)
 

Out[23]: [[ word rating movie
 0 6 2.688831 0
 1 9 2.733550 0
 2 24 3.142981 0
 3 37 3.230474 0
 4 59 3.469831 0
 5 126 3.934158 0
 6 137 3.942225 0
 7 146 4.081030 0
 8 177 4.126906 0
 9 231 4.332707 0
 10 307 4.585110 0
 11 309 4.529276 0
 12 465 4.796419 0
 13 483 4.855208 0
 14 522 4.879350 0
 15 555 4.913770 0
 16 665 5.215822 0
 17 695 5.101554 0
 18 1186 5.748822 0
 19 1341 17.806913 0
 20 1641 5.832983 0
 21 2602 6.311806 0
 22 2907 6.502033 0
 23 3807 6.713342 0
 24 4074 6.907498 0
 25 4315 6.879327 0
 26 4579 6.907498 0
 27 4985 7.028859 0
 28 5171 7.078456 0
 29 5194 7.078456 0
 30 5727 7.327352 0
 31 6861 7.442421 0
 32 7193 7.442421 0
 33 8311 7.690257 0
 34 8486 7.754796 0
 35 8487 7.659486 0
 36 8891 7.754796 0
 37 11686 16.645560 0
 38 13296 8.322780 0
 0 6 2.688831 1
 1 9 2.733550 1
 2 24 3.142981 1
 3 37 3.230474 1
 4 59 3.469831 1
 5 126 3.934158 1
 6 137 3.942225 1
 7 146 4.081030 1
 8 177 4.126906 1
 9 231 4.332707 1
 10 307 4.585110 1
 11 309 4.529276 1
 12 465 4.796419 1
 13 483 4.855208 1
 14 522 4.879350 1
 15 555 4.913770 1
 16 665 5.215822 1
 17 695 5.101554 1
 18 1186 5.748822 1
 19 1341 17.806913 1
 20 1641 5.832983 1
 21 2602 6.311806 1
 22 2907 6.502033 1
 23 3807 6.713342 1
 24 4074 6.907498 1
 25 4315 6.879327 1
 26 4579 6.907498 1
 27 4985 7.028859 1
 28 5171 7.078456 1
 29 5194 7.078456 1
 30 5727 7.327352 1
 31 6861 7.442421 1
 32 7193 7.442421 1
 33 8311 7.690257 1
 34 8486 7.754796 1
 35 8487 7.659486 1
 36 8891 7.754796 1
 37 11686 16.645560 1
 38 13296 8.322780 1
 0 3 2.594401 2
 1 4 2.644215 2
 2 27 3.209730 2
 3 45 3.259813 2
 4 60 3.500129 2
 5 211 4.374936 2
 6 234 4.322962 2
 7 262 4.452805 2
 8 307 4.585110 2
 9 454 4.858884 2
 10 919 5.384552 2
 11 1018 5.576264 2
 12 1334 5.722271 2
 13 1546 5.857676 2
 14 1789 6.008499 2
 15 2249 6.214351 2
 16 3356 6.581282 2
 17 3473 6.749274 2
 18 3593 6.667357 2
 19 4941 7.264173 2
 20 5741 7.167009 2
 21 5795 7.224168 2
 22 5874 7.185701 2
 23 6194 7.394793 2
 24 7128 7.572474 2
 25 7551 15.201291 2
 26 8029 7.600645 2
 27 12525 8.211554 2
 0 3 5.188801 3
 1 45 3.259813 3
 2 49 3.262794 3
 3 66 3.558095 3
 4 90 3.695503 3
 5 276 4.520047 3
 6 325 4.564320 3
 7 398 4.702368 3
 8 820 5.332947 3
 9 1186 5.748822 3
 10 1481 5.780715 3
 11 1812 5.968608 3
 12 1878 6.026095 3
 13 2430 6.327680 3
 14 2485 6.319711 3
 15 2771 6.343809 3
 16 3036 6.455513 3
 17 3585 6.773967 3
 18 3608 6.645134 3
 19 3761 6.749274 3
 20 5263 7.061649 3
 21 5741 7.167009 3
 22 6058 7.243970 3
 23 7234 7.722006 3
 24 9346 7.788698 3
 25 9577 7.977940 3
 26 10385 7.937118 3
 27 13055 8.265622 3
 28 14649 8.447943 3
 29 16155 8.758098 3
 30 16447 8.671087 3
 0 11 2.912974 4
 1 15 3.000057 4
 2 16 2.916664 4
 3 17 5.827527 4
 4 25 6.141629 4
 5 49 3.262794 4
 6 57 3.410125 4
 7 116 3.875985 4
 8 139 4.353599 4
 9 159 39.602020 4
 10 209 4.226477 4
 11 222 4.296378 4
 12 228 4.275609 4
 13 283 8.920389 4
 14 302 4.489037 4
 15 314 4.523992 4
 16 390 4.694525 4
 17 450 4.780969 4
 18 482 5.184732 4
 19 508 4.868135 4
 20 635 5.046746 4
 21 747 5.197569 4
 22 1177 5.572497 4
 23 1214 5.614730 4
 24 1314 5.662932 4
 25 1408 5.731043 4
 26 1487 17.664406 4
 27 1657 5.852688 4
 28 1816 5.924885 4
 29 2016 6.119041 4
 30 2501 6.280796 4
 31 2513 6.319711 4
 32 2597 6.273191 4
 33 4008 13.963212 4
 34 4065 6.825260 4
 35 4141 6.851928 4
 36 5063 7.078456 4
 37 5146 7.078456 4
 38 5746 7.284792 4
 39 6094 7.518407 4
 40 6136 7.243970 4
 41 6959 7.418324 4
 42 7100 7.442421 4
 43 7209 7.492432 4
 44 7700 15.509592 4
 45 10691 8.111471 4
 46 12222 8.322780 4
 47 13571 8.383405 4
 48 13809 16.895886 4
 49 16976 8.758098 4
 0 25 6.141629 5
 1 249 4.337069 5
 2 365 4.806854 5
 3 372 4.636846 5
 4 505 4.939387 5
 5 744 5.489567 5
 6 1491 5.804135 5
 7 2908 6.502033 5
 8 3131 6.865534 5
 9 3569 6.725177 5
 10 13394 8.322780 5], [ word rating movie
 0 4 2.644215 0
 1 5 2.789623 0
 2 7 2.673599 0
 3 10 2.841896 0
 4 24 3.142981 0
 5 53 3.357060 0
 6 86 3.719376 0
 7 266 4.406062 0
 8 287 

In [35]:
for i in range(1,500):
  thisMovie=KrescaledData['features'][i].toArray()
  #get words for this movie
  words=np.nonzero(thisMovie)
  #get weights for the words in this movie
  weights=thisMovie[thisMovie!=0]
  matrixData = pd.DataFrame()
  matrixData['word']= words[0]
  matrixData['rating']=weights
  matrixData['movie']=i
  matrix=matrix.append(ks.DataFrame(matrixData))
#matrix

Convert results from Koalas to databricks

In [37]:
df_matrix=matrix.to_spark()
df_matrix.count()

Out[25]: 17668

Split data into training and test

In [39]:
(training,test)=df_matrix.randomSplit([0.8,0.2])

#print('Training: {0}, test: {1}\n'.format(
#  training.count(), test.count())
#)
#training.show(3)
#test.show(3)

#Model

Train ALS models with different K

In [41]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

In [42]:
als = ALS()

als.setUserCol('word')\
   .setItemCol('movie')

param_grid=ParamGridBuilder()\
            .addGrid(als.rank,[8])\
            .addGrid(als.maxIter,[10])\
            .addGrid(als.regParam,[.1])\
            .build()

evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

tvs=TrainValidationSplit(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator)

In [43]:
import mlflow
import mlflow.sklearn

In [44]:
ALSmodel=tvs.fit(training)

java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:468)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:395)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:177)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:169)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:268)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:240)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:149)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:768)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:453)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:234)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:140)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:612)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:822)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:254)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1400(ManagedSelector.java:62)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:401)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:360)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:184)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:367)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:782)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:914)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:714)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:232)
	... 11 more

In [45]:
best_model=ALSmodel.bestModel

predictions=best_model.transform(test) 

predicted_ratings_df = predictions.filter(predictions.prediction != float('nan'))

rmse=evaluator.evaluate(predicted_ratings_df)

print("RMSE = " + str(rmse))
print("**Best Model**")
print(" Rank:"),best_model.rank
print(" MaxIter:"),best_model._java_obj.parent().getMaxIter()
print(" RegParam:"),best_model._java_obj.parent().getRegParam()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2587406853887826> in <module> 
 ----> 1 best_model = ALSmodel . bestModel
 2 
 3 predictions = best_model . transform ( test ) 
 4 
 5 predicted_ratings_df = predictions . filter ( predictions . prediction != float ( 'nan' ) ) 

 NameError : name 'ALSmodel' is not defined

In [46]:
display(predicted_ratings_df)

word,rating,movie,prediction
18296,8.591043987645923,1,8.526204
6,2.7893905123237235,1,2.768338
5167,7.243970339679314,1,7.15207
9299,7.860156479103131,1,7.800833
8,2.9216937094112065,1,2.8996422
4087,6.981606075211823,1,6.9289126
1078,5.696407830963301,1,5.6534147
119,3.9982578608575534,1,3.9680815
662,5.17205706442027,6,0.47961414
7066,7.4424212784031525,3,7.3923683


#Recommendations

We start by computing the 10 recommended movies for each word.

In [48]:
user_rec=best_model.recommendForAllUsers(10)

Recommendations are made for earch word. So for instance if we thing of a given word in the vocabulary, we would like to know what movies best fit such word. Lets say we want recommendations for word 7678 in the vocabulary.

In [50]:
vocabulary[1959]

Out[66]: 'moon'

In [51]:
user = user_rec.filter(user_rec.word==1959)
rec=user.select("recommendations.movie","recommendations.rating")
rec_items=rec.select("movie").toPandas().iloc[0,0]
rec_ratings=rec.select("rating").toPandas().iloc[0,0]
rec_matrix=pd.DataFrame(rec_items,columns=["movie"])
rec_matrix["ratings"]=rec_ratings
ratings_matrix_df=sqlContext.createDataFrame(rec_matrix)
display(ratings_matrix_df)

movie,ratings
1,12.43671
8,5.5398307
2,5.1620774
6,4.6677046
0,3.9043462
3,2.351719
7,2.1631382
9,-4.5959554
5,-5.635323
4,-6.1553035


We use the list of movie titles to find which movie the selected word recommends

In [53]:
movieList[ratings_matrix_df.select('movie').collect()[0].movie]

Out[68]: 'Love by the Light of the Moon'